In [1]:
!pip install torch==2.6.0 torchvision==0.21.0
!pip install transformers==4.48.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 80.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


In [2]:
!pip install accelerate==1.3.0
!pip install soundfile==0.13.1
!pip install pillow==11.1.0
!pip install scipy==1.15.2
!pip install backoff==2.2.1
!pip install peft==0.13.2

In [3]:
!pip install backoff==2.2.1

In [4]:
import os
import json
import requests
import tqdm
import pandas as pd
import torch
from PIL import Image
import soundfile
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig


model_path = "microsoft/Phi-4-multimodal-instruct"

kwargs = {}
kwargs['torch_dtype'] = torch.bfloat16

processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
print(processor.tokenizer)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype='auto',
    _attn_implementation='eager',
    low_cpu_mem_usage=True,
    device_map = "auto"
).cuda()
print("model.config._attn_implementation:", model.config._attn_implementation)

generation_config = GenerationConfig.from_pretrained(model_path, 'generation_config.json')

user_prompt = '<|user|>'
assistant_prompt = '<|assistant|>'
prompt_suffix = '<|end|>'


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:590: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


GPT2TokenizerFast(name_or_path='microsoft/Phi-4-multimodal-instruct', vocab_size=200019, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	199999: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200010: AddedToken("<|endoftext10|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200011: AddedToken("<|endoftext11|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200018: AddedToken("<|endofprompt|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	200019: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	200020: AddedToken("<|end|>", rstrip=T

/root/.cache/huggingface/modules/transformers_modules/microsoft/Phi-4-multimodal-instruct/0af439b3adb8c23fda473c4f86001dbf9a226021/speech_conformer_encoder.py:2774: FutureWarning: Please specify CheckpointImpl.NO_REENTRANT as CheckpointImpl.REENTRANT will soon be removed as the default and eventually deprecated.
  lambda i: encoder_checkpoint_wrapper(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

model.config._attn_implementation: eager


In [6]:
!pip install backoff

In [7]:
prompt = """
You are given an image of a handwritten medical prescription.
Extract the following structured information(if available):
- Doctor Name
- Clinic Address
- Patient Name
- Patient Age
- Address (if available)
- Date
- Medicine Name(s)
- Dosage
- Frequency
- Duration
Format output strictly as JSON.
    """

In [8]:
def extract_info(image_path , prompt = prompt):
    prompt = f'{user_prompt}<|image_1|>{prompt}{prompt_suffix}{assistant_prompt}'
    image = Image.open(image_path)
    inputs = processor(text=prompt, images=image, return_tensors='pt').to('cuda:0')

    output = generate(inputs)

    return output

def generate(inputs):
    generate_ids = model.generate(
        **inputs,
        max_new_tokens=1000,
        generation_config=generation_config,
    )
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1] :]
    response = processor.batch_decode(
        generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    return response

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
image_folder = '/content/drive/MyDrive/DEBJYOTI/temp/data'
output_folder = '/content/drive/MyDrive/DEBJYOTI/temp/output'
os.makedirs(output_folder, exist_ok=True)

In [11]:
results = []

image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


In [12]:
extract_info(os.path.join(image_folder, image_files[len(image_files)-1]))

'{"Doctor Name": "Dr. Venkateswara Rao", "Clinic Address": "Yashoda Hospitals, Somajiguda, Hyderabad, Telangana-500 082", "Patient Name": "Mr. Venkateswara Rao", "Patient Age": "45", "Address": "Somajiguda, Hyderabad", "Date": "18/5/2021", "Medicine Name(s)": ["Liposomal Amphotericin B", "Conventional Amphotericin B"], "Dosage": "300 mg", "Frequency": "IV OD", "Duration": "2 weeks"}'

In [13]:
image_files[0]

'104.jpg'

In [14]:
for img_file in tqdm.tqdm(image_files):
    img_path = os.path.join(image_folder, img_file)
    try:
        output = extract_info(img_path)
        results.append({
            'image': img_file,
            'extracted_info': output
        })
    except Exception as e:
        print(f"Error processing {img_file}: {e}")

100%|██████████| 134/134 [37:20<00:00, 16.72s/it]


In [15]:
with open(os.path.join(output_folder, 'extracted_data.json'), 'w') as f:
    json.dump(results, f, indent=4)

# As CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(output_folder, 'extracted_data.csv'), index=False)

In [16]:
df

,image,extracted_info
0,104.jpg,"{""Doctor Name"": ""Swo Bravm"", ""Clinic Address"":..."
1,105.jpg,"{""Doctor Name"": ""Dr. Raj Kamal Agarwal"", ""Clin..."
2,101.jpg,"{""Doctor Name"": ""Dr. Smith"", ""Clinic Address"":..."
3,103.jpg,"{""Doctor Name"": ""R.G. COMBS M.D."", ""Clinic Add..."
4,106.jpg,"{""Doctor Name"": ""Dr. Trlock"", ""Clinic Address""..."
...,...,...
129,9.jpg,"{""Doctor Name"": ""C.O. Jones"", ""Clinic Address""..."
130,90.jpg,"{""Doctor Name"": ""Dr B. Who"", ""Clinic Address"":..."
131,93.jpg,"{""Doctor Name"": ""RP"", ""Clinic Address"": ""Elinp..."
132,89.jpg,"{""Clinic Address"": ""500 - 600 W. 4TH STREET, O..."
